# Install StatsModel dan Import Package

In [ ]:
!pip install statsmodels==0.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import package
# impor library standard
import numpy as np
import pandas as pd

from statsmodels.stats.proportion import power_proportions_2indep, test_proportions_2indep
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.power import tt_ind_solve_power
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import datetime as dt
import warnings
warnings.filterwarnings('ignore')


# Load Data

Data yang akan diproses terdiri dari 3 tipe
- Sales (orders, customer, seller, order item, product, order review)
- Marketing
- AB Testing


## Wrangling Process of Sales Data Set

In [ ]:
# orders
url = 'https://github.com/shofi78/E-commerce-Project/raw/main/data/olist_orders_dataset.csv'
orders = pd.read_csv(url)
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [ ]:
#cek data null, pastikan order_id tidak null untuk digunakan sebagai primary key
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [ ]:
#cek kategori yang ada di order_status, untuk nentuin stage sales pipeline
orders.order_status.unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

In [ ]:
#cek kondisional/syarat tiap order status
#untuk status created, kolom order_purchased_timestamp tidak boleh Null
orders.loc[orders['order_status']=='created']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
7434,b5359909123fa03c50bdb0cfed07f098,438449d4af8980d107bf04571413a8e7,created,2017-12-05 01:07:52,NaN,NaN,NaN,2018-01-11 00:00:00
9238,dba5062fbda3af4fb6c33b1e040ca38f,964a6df3d9bdf60fe3e7b8bb69ed893a,created,2018-02-09 17:21:04,NaN,NaN,NaN,2018-03-07 00:00:00
21441,7a4df5d8cff4090e541401a20a22bb80,725e9c75605414b21fd8c8d5a1c2f1d6,created,2017-11-25 11:10:33,NaN,NaN,NaN,2017-12-12 00:00:00
55086,35de4050331c6c644cddc86f4f2d0d64,4ee64f4bfc542546f422da0aeb462853,created,2017-12-05 01:07:58,NaN,NaN,NaN,2018-01-08 00:00:00
58958,90ab3e7d52544ec7bc3363c82689965f,7d61b9f4f216052ba664f22e9c504ef1,created,2017-11-06 13:12:34,NaN,NaN,NaN,2017-12-01 00:00:00


In [ ]:
#untuk status processing, kolom order_purchase_timestamp dan order_approved_at tidak boleh null
orders.loc[orders['order_status']=='processing']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaN,NaN,2017-10-03 00:00:00
324,d3c8851a6651eeff2f73b0e011ac45d0,957f8e082185574de25992dc659ebbc0,processing,2016-10-05 22:44:13,2016-10-06 15:51:05,NaN,NaN,2016-12-09 00:00:00
741,6a6c7d523fd59eb5bbefc007331af717,d954782ec6c0e911292c8a80757ef28d,processing,2017-11-24 20:09:33,2017-11-24 23:15:15,NaN,NaN,2017-12-20 00:00:00
1192,745e1d4a7f8c4b548881788d4113bb1d,7198d7088442e4ddfe553353d8ddc957,processing,2017-11-17 06:34:36,2017-11-18 02:15:40,NaN,NaN,2017-12-12 00:00:00
1516,1d52ba7197c7acebbb4f826f6585536f,c9c7fe860d602373a9e93f8bfe9d877a,processing,2017-02-13 18:32:55,2017-02-13 18:43:55,NaN,NaN,2017-04-04 00:00:00
...,...,...,...,...,...,...,...,...
97400,dcdfc540e42725663242bb884c28f0a6,38972104038aa68fcc61277dbf6e7ca9,processing,2017-10-30 10:46:44,2017-10-30 11:09:55,NaN,NaN,2017-11-23 00:00:00
97666,e471815e7114cdb474064f7dbb1a8b67,092c9316ae71b2fe43e526043f351967,processing,2017-12-20 11:00:02,2017-12-20 11:10:43,NaN,NaN,2018-02-02 00:00:00
98089,10951d02d64917a34959abeb8130601e,3e6754e591ff3568ccc5bf69a649918c,processing,2018-02-13 21:02:02,2018-02-15 04:11:21,NaN,NaN,2018-03-13 00:00:00
99140,aea0db338150b526dde24f6fd953a5ed,379a02efdc6a56bd27f99b95fc2f6c06,processing,2017-12-26 21:56:13,2017-12-26 22:05:26,NaN,NaN,2018-01-26 00:00:00


In [ ]:
#order dengan status invoiced & approved, 
#order_purchase_timestamp dan order_approved_at tidak boleh null
#karena order status invoiced, approved dan processing kondisional nya sama, maka akan dijadikan
#menjadi status yang sama, yaitu 'validated'
#ataging = created -> validated ->shipped-> delivered
orders.loc[orders['order_status']=='invoiced']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaN,NaN,2018-08-21 00:00:00
455,38b7efdf33dd5561f4f5d4f6e07b0414,021e84751ba0ead75b6d314a6ead88d9,invoiced,2017-08-01 18:17:41,2017-08-01 18:32:30,NaN,NaN,2017-08-28 00:00:00
1833,51b0dccc8596ce37a930dff2d63a10a2,31bf1057b00f14804278590bbac18b1b,invoiced,2017-05-05 22:34:48,2017-05-05 22:45:12,NaN,NaN,2017-06-06 00:00:00
2116,5504eaa5a86eb25fa666cf2e6b96c701,a30be38e1ed0ffb39b318bf1d15e98d9,invoiced,2017-11-29 08:33:06,2017-11-29 08:56:23,NaN,NaN,2017-12-20 00:00:00
...,...,...,...,...,...,...,...,...
98414,41ad2ae32ee66c07ef3b0301ba02ad27,5c7f8d14a0a12198d1d07167d7d343b3,invoiced,2017-11-24 15:54:47,2017-11-24 19:33:24,NaN,NaN,2017-12-18 00:00:00
98624,9ffb325177b96c422447e3b9ee9a6270,e4079117ea0dc8bc27274e34c4796515,invoiced,2018-05-10 19:11:18,2018-05-10 19:37:27,NaN,NaN,2018-06-05 00:00:00
98845,021dac6e3120485db6a77474aa21b979,c40f7f00e91eb8a0da1e58484380a796,invoiced,2017-11-24 22:46:09,2017-11-25 01:53:42,NaN,NaN,2017-12-15 00:00:00
99031,c526fe83c01c48066394ff5d81f9ab9a,6c678c38f3a0874b7bfeeb2d5fb22876,invoiced,2018-08-07 11:11:30,2018-08-07 11:25:01,NaN,NaN,2018-08-17 00:00:00


In [ ]:
#mengganti kolom order_status invoiced, approved dan processing menjadi validated
orders['order_status']=orders['order_status'].replace(['invoiced','approved','processing'],'validated')

In [ ]:
#sanity check
orders.order_status.unique()

array(['delivered', 'validated', 'shipped', 'unavailable', 'canceled',
       'created'], dtype=object)

In [ ]:
#mengubah kolom menjadi datetime format
orders['order_purchase_timestamp']=pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at']=pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date']=pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date']=pd.to_datetime(orders['order_delivered_customer_date'])

In [ ]:
#sanity check
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  object        
dtypes: datetime64[ns](4), object(4)
memory usage: 6.1+ MB


In [ ]:
#menghitung lead time setiap stage
orders['leadtime_created_to_validated']=(orders.order_approved_at-orders.order_purchase_timestamp).dt.total_seconds() / (24 * 60 * 60)
orders['leadtime_validated_to_shipped']=(orders.order_delivered_carrier_date-orders.order_approved_at).dt.total_seconds() / (24 * 60 * 60)
orders['leadtime_shipped_to_delivered']=(orders.order_delivered_customer_date-orders.order_delivered_carrier_date).dt.total_seconds() / (24 * 60 * 60)
orders['leadtime_created_to_shipped']=(orders.order_delivered_carrier_date-orders.order_purchase_timestamp).dt.total_seconds() / (24 * 60 * 60)

In [ ]:
#cekiceki
orders.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,leadtime_created_to_validated,leadtime_validated_to_shipped,leadtime_shipped_to_delivered,leadtime_created_to_shipped
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,0.007431,2.366493,6.062650,2.373924
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.279745,0.462882,12.039410,1.742627
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,0.011505,0.204595,9.178113,0.216100
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,0.012419,3.745833,9.450498,3.758252
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,0.042940,0.893113,1.937824,0.936053


In [ ]:
from google.colab import files
orders.to_excel('orders.xlsx')
files.download('orders.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# customers
url = 'https://github.com/shofi78/E-commerce-Project/raw/main/data/olist_customers_dataset.csv'
customers = pd.read_csv(url)
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [ ]:
# sellers
url = 'https://github.com/shofi78/E-commerce-Project/raw/main/data/olist_sellers_dataset.csv'
sellers = pd.read_csv(url)
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [ ]:
from google.colab import files
sellers.to_excel('sellers.xlsx')
files.download('sellers.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# products
url = 'https://github.com/shofi78/E-commerce-Project/raw/main/data/olist_products_dataset.csv'
products = pd.read_csv(url)
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [ ]:
# order_items
url ='https://github.com/shofi78/E-commerce-Project/raw/main/data/olist_order_items_dataset.csv'
order_items = pd.read_csv(url)
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [ ]:
order_items['order_id'].value_counts()

8272b63d03f5f79c56e9e4120aec44ef    21
1b15974a0141d54e36626dca3fdc731a    20
ab14fdcfbe524636d65ee38360e22ce8    20
9ef13efd6949e4573a18964dd1bbe7f5    15
428a2f660dc84138d969ccd69a0ab6d5    15
                                    ..
5a0911d70c1f85d3bed0df1bf693a6dd     1
5a082b558a3798d3e36d93bfa8ca1eae     1
5a07264682e0b8fbb3f166edbbffc6e8     1
5a071192a28951b76774e5a760c8c9b7     1
fffe41c64501cc87c801fd61db3f6244     1
Name: order_id, Length: 98666, dtype: int64

In [ ]:
#cek perhitungan jumlah orderan. Dari hasil yang ditampilkan, order dibagi per item. Untuk tiap item yang sama dari seller yang sama,
#price dikalikan jumlah produk yang dibeli, dan freight value diasumsikan tidak bertambah dengan bertambahnya jumlah item
order_items.loc[order_items['order_id'] == '8272b63d03f5f79c56e9e4120aec44ef']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
57297,8272b63d03f5f79c56e9e4120aec44ef,1,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57298,8272b63d03f5f79c56e9e4120aec44ef,2,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57299,8272b63d03f5f79c56e9e4120aec44ef,3,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57300,8272b63d03f5f79c56e9e4120aec44ef,4,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57301,8272b63d03f5f79c56e9e4120aec44ef,5,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57302,8272b63d03f5f79c56e9e4120aec44ef,6,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57303,8272b63d03f5f79c56e9e4120aec44ef,7,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57304,8272b63d03f5f79c56e9e4120aec44ef,8,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57305,8272b63d03f5f79c56e9e4120aec44ef,9,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57306,8272b63d03f5f79c56e9e4120aec44ef,10,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89


In [ ]:
#untuk melihat per order_id membeli berapa tipe produk, total pembelian per tipe produk & ongkir per tipe produk
order_item_tipe =order_items.groupby(['order_id','product_id','seller_id','freight_value']).agg({'product_id': 'count', 'price': 'sum'}).rename(columns={'product_id':'COUNT_PRODUCT'}).reset_index()

In [ ]:
#sanity check apakah perhitungan sudah benar
order_item_tipe.loc[order_item_tipe['order_id']=='8272b63d03f5f79c56e9e4120aec44ef']

,order_id,product_id,seller_id,freight_value,COUNT_PRODUCT,price
52027,8272b63d03f5f79c56e9e4120aec44ef,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,7.89,10,12.0
52028,8272b63d03f5f79c56e9e4120aec44ef,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,7.89,10,12.0
52029,8272b63d03f5f79c56e9e4120aec44ef,79ce45dbc2ea29b22b5a261bbb7b7ee7,2709af9587499e95e803a6498a5a56e9,6.57,1,7.8


In [ ]:
#menampilkan kategori produk ke dalam tabel order_item_tipe
order_item_tipe = order_item_tipe.merge(products, how='left', on='product_id')
order_item_tipe.head(5)

,order_id,product_id,seller_id,freight_value,COUNT_PRODUCT,price,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,13.29,1,58.90,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,19.93,1,239.90,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,17.87,1,199.00,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.79,1,12.99,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,18.14,1,199.90,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0


In [ ]:
#drop kolom yang tidak digunakan
order_item_tipe.drop(['product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm'],axis=1,inplace=True)

In [ ]:
#cekiceki
order_item_tipe.head(3)

,order_id,product_id,seller_id,freight_value,COUNT_PRODUCT,price,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,13.29,1,58.9,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,19.93,1,239.9,pet_shop
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,17.87,1,199.0,moveis_decoracao


In [ ]:
df_order=order_item_tipe.groupby(['order_id','seller_id']).agg({'product_id': 'count', 'price': 'sum', 'freight_value':'sum','COUNT_PRODUCT':'sum'}).rename(columns={'product_id':'COUNT_TYPE','price':'SUM_PRICE','freight_value':'SUM_FREIGHT'}).reset_index()

In [ ]:
#sanity check
order_item_tipe.loc[order_item_tipe['order_id']=='8272b63d03f5f79c56e9e4120aec44ef']

,order_id,product_id,seller_id,freight_value,COUNT_PRODUCT,price,product_category_name
52027,8272b63d03f5f79c56e9e4120aec44ef,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,7.89,10,12.0,beleza_saude
52028,8272b63d03f5f79c56e9e4120aec44ef,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,7.89,10,12.0,beleza_saude
52029,8272b63d03f5f79c56e9e4120aec44ef,79ce45dbc2ea29b22b5a261bbb7b7ee7,2709af9587499e95e803a6498a5a56e9,6.57,1,7.8,beleza_saude


In [ ]:
total_sales=orders.merge(order_item_tipe, on=['order_id'], how='left').groupby(['order_id'],as_index=False)['price'].sum()

In [ ]:
dataframe=[orders, total_sales]

In [ ]:
from functools import reduce

In [ ]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['order_id'],
                                            how='left'), dataframe)
df_merged

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,leadtime_created_to_validated,leadtime_validated_to_shipped,leadtime_shipped_to_delivered,leadtime_created_to_shipped,price
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,0.007431,2.366493,6.062650,2.373924,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.279745,0.462882,12.039410,1.742627,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,0.011505,0.204595,9.178113,0.216100,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,0.012419,3.745833,9.450498,3.758252,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,0.042940,0.893113,1.937824,0.936053,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,0.000000,1.058310,7.159699,1.058310,72.00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,0.008090,1.425058,20.760579,1.433148,174.90
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,0.012188,1.241782,23.605451,1.253970,205.99
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,0.005486,3.749097,13.331840,3.754583,359.98


In [ ]:
df_merged

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,leadtime_created_to_validated,leadtime_validated_to_shipped,leadtime_shipped_to_delivered,leadtime_created_to_shipped,price
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,0.007431,2.366493,6.062650,2.373924,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.279745,0.462882,12.039410,1.742627,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,0.011505,0.204595,9.178113,0.216100,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,0.012419,3.745833,9.450498,3.758252,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,0.042940,0.893113,1.937824,0.936053,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,0.000000,1.058310,7.159699,1.058310,72.00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,0.008090,1.425058,20.760579,1.433148,174.90
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,0.012188,1.241782,23.605451,1.253970,205.99
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,0.005486,3.749097,13.331840,3.754583,359.98


In [ ]:
from google.colab import files
df_merged.to_excel('df_merged.xlsx')
files.download('df_merged.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
order_item_tipe.to_excel('order_item_tipe.xlsx')
files.download('order_item_tipe.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# order_reviews
url = 'https://github.com/shofi78/E-commerce-Project/raw/main/data/olist_order_reviews_dataset.csv'
order_reviews = pd.read_csv(url)
order_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [ ]:
#cek tipe produk yang statusnya unavailable
unavail_product = orders.loc[orders['order_status']=='unavailable'].merge(order_item_tipe, how='left', on='order_id')
unavail_product

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,leadtime_created_to_validated,leadtime_validated_to_shipped,leadtime_shipped_to_delivered,leadtime_created_to_shipped,product_id,seller_id,freight_value,COUNT_PRODUCT,price,product_category_name
0,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaT,NaT,2017-12-05 00:00:00,0.012141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaT,NaT,2018-02-16 00:00:00,0.119595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,NaT,NaT,2017-09-05 00:00:00,2.275880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,d57e15fb07fd180f06ab3926b39edcd2,470b93b3f1cde85550fc74cd3a476c78,unavailable,2018-01-08 19:39:03,2018-01-09 07:26:08,NaT,NaT,2018-02-06 00:00:00,0.491030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2f634e2cebf8c0283e7ef0989f77d217,7353b0fb8e8d9675e3a704c60ca44ebe,unavailable,2017-09-27 20:55:33,2017-09-28 01:32:50,NaT,NaT,2017-10-27 00:00:00,0.192558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,6cf5c427fb11bc5fe8abefa86aa7ba0b,91ac14a1e50bc022eeecf0c2ba37006a,unavailable,2017-10-17 10:53:48,2017-10-19 03:12:43,NaT,NaT,2017-10-27 00:00:00,1.679803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,1c7c8ab324927f5b6e52338bd110bef4,95b1767df51d6fa09402267d8de314df,unavailable,2017-12-23 18:48:09,2017-12-23 18:54:22,NaT,NaT,2018-01-19 00:00:00,0.004317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,aaab15da689073f8f9aa978a390a69d1,df20748206e4b865b2f14a5eabbfcf34,unavailable,2018-01-16 14:27:59,2018-01-17 03:37:34,NaT,NaT,2018-02-06 00:00:00,0.548322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaT,NaT,2017-09-15 00:00:00,4.969942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Wrangling Process of Marketing Data Set

In [ ]:
import pandas as pd
# Marketing
url = 'https://github.com/shofi78/E-commerce-Project/raw/main/data/marketing_data.xlsx'
marketing = pd.read_excel(url, parse_dates=['Dt_Customer']).sort_values('Dt_Customer').reset_index()
marketing.rename(columns={" Income ":"Income"}, inplace=True)
marketing

,index,ID,Year_Birth,Education,Marital_Status,Income,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,NumWebPurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2
0,1642,6653,1970,PhD,Married,"$55,158.00",2012-07-30,72,293,0,87,4,7,7,0,0,0,0,0
1,837,6181,1950,2n Cycle,Together,"$52,203.00",2012-07-31,36,488,21,238,56,8,6,0,0,0,0,0
2,2222,10001,1985,2n Cycle,Together,"$7,500.00",2012-08-01,98,5,17,17,13,2,9,0,0,0,0,0
3,1509,2611,1959,Master,Together,"$82,576.00",2012-08-01,66,1206,55,445,168,2,1,0,0,1,0,0
4,1398,7118,1957,Graduation,Married,"$73,803.00",2012-08-01,61,833,80,363,52,9,6,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,1617,8093,1969,Master,Married,"$79,734.00",2014-06-28,72,572,8,232,23,4,1,0,0,0,1,0
2236,1616,1371,1976,Graduation,Single,"$79,941.00",2014-06-28,72,123,164,266,227,2,1,0,0,0,0,0
2237,2201,6722,1954,PhD,Married,"$70,421.00",2014-06-28,98,479,28,136,75,10,6,0,0,0,0,0
2238,1122,7300,1952,Graduation,Divorced,"$69,142.00",2014-06-29,50,448,4,34,6,8,5,0,1,0,0,0


In [ ]:
marketing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   index              2240 non-null   int64         
 1   ID                 2240 non-null   int64         
 2   Year_Birth         2240 non-null   int64         
 3   Education          2240 non-null   object        
 4   Marital_Status     2240 non-null   object        
 5   Income             2216 non-null   object        
 6   Dt_Customer        2240 non-null   datetime64[ns]
 7   Recency            2240 non-null   int64         
 8   MntWines           2240 non-null   int64         
 9   MntFruits          2240 non-null   int64         
 10  MntMeatProducts    2240 non-null   int64         
 11  MntFishProducts    2240 non-null   int64         
 12  NumWebPurchases    2240 non-null   int64         
 13  NumWebVisitsMonth  2240 non-null   int64         
 14  Accepted

In [ ]:
marketing.isna().sum()

index                 0
ID                    0
Year_Birth            0
Education             0
Marital_Status        0
Income               24
Dt_Customer           0
Recency               0
MntWines              0
MntFruits             0
MntMeatProducts       0
MntFishProducts       0
NumWebPurchases       0
NumWebVisitsMonth     0
AcceptedCmp3          0
AcceptedCmp4          0
AcceptedCmp5          0
AcceptedCmp1          0
AcceptedCmp2          0
dtype: int64

In [ ]:
# Imputasi nilai NaN dengan methode ffill
marketing.ffill(axis=0, inplace=True)

In [ ]:
# Sanity check
marketing.isna().sum()

index                0
ID                   0
Year_Birth           0
Education            0
Marital_Status       0
Income               0
Dt_Customer          0
Recency              0
MntWines             0
MntFruits            0
MntMeatProducts      0
MntFishProducts      0
NumWebPurchases      0
NumWebVisitsMonth    0
AcceptedCmp3         0
AcceptedCmp4         0
AcceptedCmp5         0
AcceptedCmp1         0
AcceptedCmp2         0
dtype: int64

In [ ]:
marketing = marketing.drop_duplicates()

In [ ]:
# Menghilangkan tanda dollar 
marketing["Income"] = marketing["Income"].replace('[^\d\.]+', '', regex=True).astype(float)

In [ ]:
marketing.describe()

,index,ID,Year_Birth,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,NumWebPurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2
count,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000
mean,1119.500000,5592.159821,1968.805804,52284.982143,49.109375,303.935714,26.302232,166.950000,37.525446,4.084821,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393
std,646.776623,3246.662198,11.984069,25214.196566,28.962453,336.597393,39.773434,225.715373,54.628979,2.778714,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976
min,0.000000,0.000000,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,559.750000,2828.250000,1959.000000,35335.500000,24.000000,23.750000,1.000000,16.000000,3.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1119.500000,5458.500000,1970.000000,51381.500000,49.000000,173.500000,8.000000,67.000000,12.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1679.250000,8427.750000,1977.000000,68468.250000,74.000000,504.250000,33.000000,232.000000,50.000000,6.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2239.000000,11191.000000,1996.000000,666666.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,27.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
marketing['Income_Category'] = pd.cut(marketing['Income'], bins=[0, 50000, 100000, 1000000], include_lowest=True, labels=['Low', 'Mid', 'High'])
marketing['Income_Category'].value_counts()

Mid     1154
Low     1072
High      14
Name: Income_Category, dtype: int64

In [ ]:
marketing['Marital_Status'].unique()

array(['Married', 'Together', 'Divorced', 'Widow', 'Single', 'YOLO',
       'Absurd', 'Alone'], dtype=object)

In [ ]:
marketing['Marital_Status'].replace('Together', value = 'Married', inplace=True )
marketing['Marital_Status'].replace(['Divorced', 'Widow', 'Single', 'YOLO', 'Absurd', 'Alone'], value = 'Single', inplace=True )
marketing['Marital_Status'].value_counts()

Married    1444
Single      796
Name: Marital_Status, dtype: int64

In [ ]:
marketing['Generation'] = marketing['Year_Birth'].copy()

In [ ]:
marketing['Generation'].replace(range(1890,1946), value = 'Traditional', inplace=True)
marketing['Generation'].replace(range(1946,1965), value = 'Boomers', inplace=True)
marketing['Generation'].replace(range(1965,1980), value = 'Gen X', inplace=True)
marketing['Generation'].replace(range(1980,1997), value = 'Millennials', inplace=True)
marketing['Generation'].value_counts()

Gen X          1030
Boomers         759
Millennials     424
Traditional      27
Name: Generation, dtype: int64

In [ ]:
marketing

,index,ID,Year_Birth,Education,Marital_Status,Income,Dt_Customer,Recency,MntWines,MntFruits,...,NumWebPurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Income_Category,Generation,Total_Price
0,1642,6653,1970,PhD,Married,55158.0,2012-07-30,72,293,0,...,7,7,0,0,0,0,0,Mid,Gen X,384
1,837,6181,1950,2n Cycle,Married,52203.0,2012-07-31,36,488,21,...,8,6,0,0,0,0,0,Mid,Boomers,803
2,2222,10001,1985,2n Cycle,Married,7500.0,2012-08-01,98,5,17,...,2,9,0,0,0,0,0,Low,Millennials,52
3,1509,2611,1959,Master,Married,82576.0,2012-08-01,66,1206,55,...,2,1,0,0,1,0,0,Mid,Boomers,1874
4,1398,7118,1957,Graduation,Married,73803.0,2012-08-01,61,833,80,...,9,6,1,0,0,0,0,Mid,Boomers,1328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,1617,8093,1969,Master,Married,79734.0,2014-06-28,72,572,8,...,4,1,0,0,0,1,0,Mid,Gen X,835
2236,1616,1371,1976,Graduation,Single,79941.0,2014-06-28,72,123,164,...,2,1,0,0,0,0,0,Mid,Gen X,780
2237,2201,6722,1954,PhD,Married,70421.0,2014-06-28,98,479,28,...,10,6,0,0,0,0,0,Mid,Boomers,718
2238,1122,7300,1952,Graduation,Single,69142.0,2014-06-29,50,448,4,...,8,5,0,1,0,0,0,Mid,Boomers,492


In [ ]:
# Menambahkan kolom Total Price
marketing["Total_Price"] = marketing["MntWines"] + marketing["MntFruits"] + marketing["MntMeatProducts"] + marketing["MntFishProducts"]
marketing

,index,ID,Year_Birth,Education,Marital_Status,Income,Dt_Customer,Recency,MntWines,MntFruits,...,NumWebPurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Income_Category,Generation,Total_Price
0,1642,6653,1970,PhD,Married,55158.0,2012-07-30,72,293,0,...,7,7,0,0,0,0,0,Mid,Gen X,384
1,837,6181,1950,2n Cycle,Married,52203.0,2012-07-31,36,488,21,...,8,6,0,0,0,0,0,Mid,Boomers,803
2,2222,10001,1985,2n Cycle,Married,7500.0,2012-08-01,98,5,17,...,2,9,0,0,0,0,0,Low,Millennials,52
3,1509,2611,1959,Master,Married,82576.0,2012-08-01,66,1206,55,...,2,1,0,0,1,0,0,Mid,Boomers,1874
4,1398,7118,1957,Graduation,Married,73803.0,2012-08-01,61,833,80,...,9,6,1,0,0,0,0,Mid,Boomers,1328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,1617,8093,1969,Master,Married,79734.0,2014-06-28,72,572,8,...,4,1,0,0,0,1,0,Mid,Gen X,835
2236,1616,1371,1976,Graduation,Single,79941.0,2014-06-28,72,123,164,...,2,1,0,0,0,0,0,Mid,Gen X,780
2237,2201,6722,1954,PhD,Married,70421.0,2014-06-28,98,479,28,...,10,6,0,0,0,0,0,Mid,Boomers,718
2238,1122,7300,1952,Graduation,Single,69142.0,2014-06-29,50,448,4,...,8,5,0,1,0,0,0,Mid,Boomers,492


In [ ]:
# save to excel dan download untuk kebutuhan visualisasi
from google.colab import files
marketing.to_excel("marketing_by_segmentation.xlsx") 
files.download("marketing_by_segmentation.xlsx")
 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Membuat RFM Analysis

In [ ]:
rfm = marketing[["ID","Recency","NumWebPurchases","Total_Price"]].sort_values("ID")
col_list = ['ID','Recency','Frequency','Monetary']
rfm.columns = col_list
rfm

,ID,Recency,Frequency,Monetary
904,0,66,3,1057
2195,1,0,7,540
64,9,86,3,84
1158,13,57,1,24
1303,17,81,6,933
...,...,...,...,...
1927,11178,69,1,27
1277,11181,85,0,6
960,11187,23,2,32
1844,11188,84,2,52


In [ ]:
# Calculate RFM Score
rfm["R"] = pd.qcut(rfm["Recency"],5,labels=[5,4,3,2,1])
rfm["F"] = pd.qcut(rfm["Frequency"],5,labels=[1,2,3,4,5])
rfm["M"] = pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])
rfm["RFM_Score"] = rfm["R"].astype(str) + rfm["F"].astype(str) + rfm["M"].astype(str)


In [ ]:
rfm

,ID,Recency,Frequency,Monetary,R,F,M,RFM_Score
904,0,66,3,1057,2,2,5,225
2195,1,0,7,540,5,5,4,554
64,9,86,3,84,1,2,2,122
1158,13,57,1,24,3,1,1,311
1303,17,81,6,933,1,4,4,144
...,...,...,...,...,...,...,...,...
1927,11178,69,1,27,2,1,1,211
1277,11181,85,0,6,1,1,1,111
960,11187,23,2,32,4,1,1,411
1844,11188,84,2,52,1,1,2,112


In [ ]:
# Customer Segmentation by Recency and Frequency
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [ ]:
# Menambahkan kolom Segment
rfm['Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,ID,Recency,Frequency,Monetary,R,F,M,RFM_Score,Segment
904,0,66,3,1057,2,2,5,225,Hibernating
2195,1,0,7,540,5,5,4,554,Champions
64,9,86,3,84,1,2,2,122,Hibernating
1158,13,57,1,24,3,1,1,311,About to Sleep
1303,17,81,6,933,1,4,4,144,At Risk


In [ ]:
# Merge data marketing dan data RFM
df_marketing = pd.merge(marketing, rfm, how="left", on=["ID"])
df_marketing.sort_values("ID")

,index,ID,Year_Birth,Education,Marital_Status,Income,Dt_Customer,Recency_x,MntWines,MntFruits,...,Generation,Total_Price,Recency_y,Frequency,Monetary,R,F,M,RFM_Score,Segment
904,1503,0,1985,Graduation,Married,70951.0,2013-05-04,66,239,10,...,Millennials,1057,66,3,1057,2,2,5,225,Hibernating
2195,1,1,1961,Graduation,Single,57091.0,2014-06-15,0,464,5,...,Boomers,540,0,7,540,5,5,4,554,Champions
64,1956,9,1975,Master,Single,46098.0,2012-08-18,86,57,0,...,Gen X,84,86,3,84,1,2,2,122,Hibernating
1158,1311,13,1947,PhD,Single,25358.0,2013-07-22,57,19,0,...,Boomers,24,57,1,24,3,1,1,311,About to Sleep
1303,1834,17,1971,PhD,Married,60491.0,2013-09-06,81,637,47,...,Gen X,933,81,6,933,1,4,4,144,At Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,1554,11178,1972,Master,Single,42394.0,2014-03-23,69,15,2,...,Gen X,27,69,1,27,2,1,1,211,Hibernating
1277,1925,11181,1949,PhD,Married,156924.0,2013-08-29,85,2,1,...,Boomers,6,85,0,6,1,1,1,111,Hibernating
960,529,11187,1978,Basic,Single,26487.0,2013-05-20,23,2,8,...,Gen X,32,23,2,32,4,1,1,411,Promising
1844,1897,11188,1957,Graduation,Married,26091.0,2014-02-25,84,15,10,...,Boomers,52,84,2,52,1,1,2,112,Hibernating


In [ ]:
# extract segment
grade = rfm['Segment'].unique()
grade

array(['Hibernating', 'Champions', 'About to Sleep', 'At Risk',
       'Promising', "Can't Loose", 'New Customers', 'Loyal Customers',
       'Potential Loyalists', 'Need Attention'], dtype=object)

In [ ]:
# Adding Grade to RFM Segment
df_grade = pd.DataFrame({'Segment': ['Hibernating', 'At Risk', "Can't Loose", 'About to Sleep', 'Need Attention', 'Loyal Customers', 'Promising' , 'New Customers', 'Potential Loyalists',  'Champions' ],
                           'Grade': [10,9,8,7,6,5,4,3,2,1]})
df_grade

,Segment,Grade
0,Hibernating,10
1,At Risk,9
2,Can't Loose,8
3,About to Sleep,7
4,Need Attention,6
5,Loyal Customers,5
6,Promising,4
7,New Customers,3
8,Potential Loyalists,2
9,Champions,1


In [ ]:
# Merge data marketing dan data grade
df_marketing = pd.merge(df_marketing, df_grade, how="left", on=["Segment"])
df_marketing.sort_values("ID")

,index,ID,Year_Birth,Education,Marital_Status,Income,Dt_Customer,Recency_x,MntWines,MntFruits,...,Total_Price,Recency_y,Frequency,Monetary,R,F,M,RFM_Score,Segment,Grade
904,1503,0,1985,Graduation,Married,70951.0,2013-05-04,66,239,10,...,1057,66,3,1057,2,2,5,225,Hibernating,10
2195,1,1,1961,Graduation,Single,57091.0,2014-06-15,0,464,5,...,540,0,7,540,5,5,4,554,Champions,1
64,1956,9,1975,Master,Single,46098.0,2012-08-18,86,57,0,...,84,86,3,84,1,2,2,122,Hibernating,10
1158,1311,13,1947,PhD,Single,25358.0,2013-07-22,57,19,0,...,24,57,1,24,3,1,1,311,About to Sleep,7
1303,1834,17,1971,PhD,Married,60491.0,2013-09-06,81,637,47,...,933,81,6,933,1,4,4,144,At Risk,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,1554,11178,1972,Master,Single,42394.0,2014-03-23,69,15,2,...,27,69,1,27,2,1,1,211,Hibernating,10
1277,1925,11181,1949,PhD,Married,156924.0,2013-08-29,85,2,1,...,6,85,0,6,1,1,1,111,Hibernating,10
960,529,11187,1978,Basic,Single,26487.0,2013-05-20,23,2,8,...,32,23,2,32,4,1,1,411,Promising,4
1844,1897,11188,1957,Graduation,Married,26091.0,2014-02-25,84,15,10,...,52,84,2,52,1,1,2,112,Hibernating,10


In [ ]:
# save to excel dan download untuk kebutuhan visualisasi
from google.colab import files
df_marketing.to_excel("df_marketing.xlsx") 
files.download("df_marketing.xlsx")
 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Wrangling Process of AB Testing



1.   Significance Level : 0,05



In [ ]:
# AB Testing
url = 'https://github.com/shofi78/E-commerce-Project/raw/main/data/ab.csv'
ab = pd.read_csv(url, parse_dates=['timestamp'])
ab 

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


There are 294478 rows in the DataFrame, each representing a user session, as well as 5 columns :

*  user_id : Sample of A/B testing. The list is random
*  timestamp : Timestamp where the A/B testing was taken
*  group : Treatment and Control
*  landing_page : Old Page and New Page. New Page for Treatment, Old Page for Control.
*  converted : Binary 0 and 1,
        0 = not converted - new landing page did not affect user conversion, 
        1 = converted - new landing page affect user conversion.

In [ ]:
# Melihat data di masing masing landing page
pd.crosstab(ab['group'], ab['landing_page'])

landing_page,new_page,old_page
group,,
control,1928,145274
treatment,145311,1965


In [ ]:
# Check dan drop data yang duplikat
ab.drop_duplicates()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


In [ ]:
# Memfilter sesuai group dan landing page yang seharusnya
ab_control = ab[ab["group"]=="control"][ab["landing_page"]=="old_page"]
ab_treatment = ab[ab["group"]=="treatment"][ab["landing_page"]=="new_page"]


In [ ]:
# Menggabungkan hasil filter
ab_testing = pd.concat([ab_control, ab_treatment], ignore_index=True)
ab_testing

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,864975,2017-01-21 01:52:26.210827,control,old_page,1
3,936923,2017-01-10 15:20:49.083499,control,old_page,0
4,719014,2017-01-17 01:48:29.539573,control,old_page,0
...,...,...,...,...,...
290580,677163,2017-01-03 19:41:51.902148,treatment,new_page,0
290581,925675,2017-01-07 20:38:26.346410,treatment,new_page,0
290582,643562,2017-01-02 19:20:05.460595,treatment,new_page,0
290583,822004,2017-01-04 03:36:46.071379,treatment,new_page,0


In [ ]:
ab_testing.drop_duplicates()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,864975,2017-01-21 01:52:26.210827,control,old_page,1
3,936923,2017-01-10 15:20:49.083499,control,old_page,0
4,719014,2017-01-17 01:48:29.539573,control,old_page,0
...,...,...,...,...,...
290580,677163,2017-01-03 19:41:51.902148,treatment,new_page,0
290581,925675,2017-01-07 20:38:26.346410,treatment,new_page,0
290582,643562,2017-01-02 19:20:05.460595,treatment,new_page,0
290583,822004,2017-01-04 03:36:46.071379,treatment,new_page,0


### Formulasi Hipotesis
Menggunakan notasi conversion rate sebagai suatu probabilitas pada distribusi binomial, hipotesis dapat diformulasikan seperti berikut ini:

*  H0 : Rata-rata conversion rate kedua landing page sama saja, pA=pB 
*  H1 : Rata-rata conversion rate kedua landing page berbeda,  pA≠pB 


### Menganalisis Hasil Eksperimen
Kita akan gunakan fungsi test_proportions_2indep dari library statsmodels untuk mengevaluasi hasil eksperimen.

In [ ]:
temp_case_1_result = ab_testing.groupby("group").agg({"converted":['count','sum']}).reset_index().values
df_case_1_result = pd.DataFrame(temp_case_1_result,columns = ["group","visit","converted"] )
df_case_1_result["conversion_rate"] = df_case_1_result["converted"]/df_case_1_result["visit"]
df_case_1_result

,group,visit,converted,conversion_rate
0,control,145274,17489,0.120386
1,treatment,145311,17264,0.118807


In [ ]:
# Menggunakan test_proportions_2indep dari library statsmodels untuk mengevaluasi hasil eksperimen
analysis_case_1_result = test_proportions_2indep(count1=df_case_1_result["converted"][1], 
                                                 nobs1=df_case_1_result["visit"][1], 
                                                 count2=df_case_1_result["converted"][0], 
                                                 nobs2=df_case_1_result["visit"][0], 
                                                 alternative="two-sided")
print(analysis_case_1_result)

statistic = -1.3115777321660258
pvalue = 0.18966265043200425
compare = diff
method = agresti-caffo
diff = -0.0015790565976871451
ratio = 0.9868834199684143
odds_ratio = 0.9851149705891606
variance = 1.449426178689978e-06
alternative = two-sided
value = 0
tuple = (-1.3115777321660258, 0.18966265043200425)


In [ ]:
# Menggunakan power_proportions_2indep dari library statsmodels untuk mengetahui power hasil eksperimen

test_split = df_case_1_result['visit'][1] / (df_case_1_result['visit'][1] + df_case_1_result['visit'][0])
relative_effect = (df_case_1_result["converted"][1] - df_case_1_result["converted"][0])/df_case_1_result["converted"][0]  

df_case_1_result['power'] = power_proportions_2indep(diff=df_case_1_result["conversion_rate"][0]*relative_effect,
                                               prop2=df_case_1_result["conversion_rate"][0], 
                                               nobs1=df_case_1_result["visit"][1],
                                               ratio=(1 - test_split)/test_split,
                                               alpha=0.05,
                                               alternative='two-sided', 
                                     return_results=False)

df_case_1_result

,group,visit,converted,conversion_rate,power
0,control,145274,17489,0.120386,0.250905
1,treatment,145311,17264,0.118807,0.250905


#### Ringkasan Uji Hipotesis
Mengetahui bahwa p-value > 0.05 (significance level yang dipilih). Kita **tidak menolak hipotesis nol ( H0 )** dan kita menyimpulkan bahwa:

Rata-rata Conversion Rate (CVR) kedua landing page tidak berbeda secara signifikan, bahwa varian tes (New Page) memiliki CVR sebesar 11.8% dengan ratio sebesar 98% relatif terhadap CVR kontrol (Old Page).

Keputusan bisnis yang diambil adalah baik Old Page ataupun New Page secara statistik tidak berpengaruh terhadap CVR. Namun, perlu dilakukan analisis dari aspek bisnis lain untuk menentukan penggunaan Landing Page.

Meskipun kita melihat perbedaan pada nilai rerata, karena p-value melebihi significance level 0.05. Kita gagal menolak hipotesis nol. Artinya tidak terdapat cukup bukti untuk mengatakan adanya perbedaan positif dari campaign  ini terhadap peningkatan nilai rerata besaran belanja bulanan.

## Bagaimana jika kita buat analisis replot berapa MDE yang dapat terdeteksi jika powernya minimal 80%

### Mari siapkan dan olah datanya terlebih dahulu

In [ ]:
# Menambahkan kolom date
ab_testing['date'] = pd.to_datetime(ab_testing['timestamp']).dt.date

In [ ]:
ab_test = ab_testing[['group', 'converted', 'date']]
ab_test

,group,converted,date
0,control,0,2017-01-21
1,control,0,2017-01-12
2,control,1,2017-01-21
3,control,0,2017-01-10
4,control,0,2017-01-17
...,...,...,...
290580,treatment,0,2017-01-03
290581,treatment,0,2017-01-07
290582,treatment,0,2017-01-02
290583,treatment,0,2017-01-04


In [ ]:
nilai = ab_test.groupby(['date', 'group']).agg(['count', 'sum']).unstack().reset_index().values
nilai

array([[datetime.date(2017, 1, 2), 2859, 2853, 359, 342],
       [datetime.date(2017, 1, 3), 6590, 6618, 750, 753],
       [datetime.date(2017, 1, 4), 6578, 6541, 802, 763],
       [datetime.date(2017, 1, 5), 6427, 6505, 792, 748],
       [datetime.date(2017, 1, 6), 6606, 6747, 762, 833],
       [datetime.date(2017, 1, 7), 6604, 6609, 799, 768],
       [datetime.date(2017, 1, 8), 6687, 6700, 795, 809],
       [datetime.date(2017, 1, 9), 6628, 6615, 793, 781],
       [datetime.date(2017, 1, 10), 6654, 6696, 751, 846],
       [datetime.date(2017, 1, 11), 6688, 6673, 795, 768],
       [datetime.date(2017, 1, 12), 6522, 6637, 796, 812],
       [datetime.date(2017, 1, 13), 6552, 6508, 766, 724],
       [datetime.date(2017, 1, 14), 6548, 6600, 830, 787],
       [datetime.date(2017, 1, 15), 6714, 6549, 809, 743],
       [datetime.date(2017, 1, 16), 6591, 6545, 803, 780],
       [datetime.date(2017, 1, 17), 6617, 6538, 813, 832],
       [datetime.date(2017, 1, 18), 6482, 6603, 809, 824],
     

In [ ]:
#dataframe baru
daily_ab_testing = pd.DataFrame(nilai, columns=['date', 'control_visit', 'treatment_visit', 'control_converted', 'treatment_converted'])
daily_ab_testing

,date,control_visit,treatment_visit,control_converted,treatment_converted
0,2017-01-02,2859,2853,359,342
1,2017-01-03,6590,6618,750,753
2,2017-01-04,6578,6541,802,763
3,2017-01-05,6427,6505,792,748
4,2017-01-06,6606,6747,762,833
5,2017-01-07,6604,6609,799,768
6,2017-01-08,6687,6700,795,809
7,2017-01-09,6628,6615,793,781
8,2017-01-10,6654,6696,751,846
9,2017-01-11,6688,6673,795,768


In [ ]:
daily_ab_testing = daily_ab_testing.set_index('date').cumsum().reset_index() #cummulative sum
daily_ab_testing

,date,control_visit,treatment_visit,control_converted,treatment_converted
0,2017-01-02,2859,2853,359,342
1,2017-01-03,9449,9471,1109,1095
2,2017-01-04,16027,16012,1911,1858
3,2017-01-05,22454,22517,2703,2606
4,2017-01-06,29060,29264,3465,3439
5,2017-01-07,35664,35873,4264,4207
6,2017-01-08,42351,42573,5059,5016
7,2017-01-09,48979,49188,5852,5797
8,2017-01-10,55633,55884,6603,6643
9,2017-01-11,62321,62557,7398,7411


In [ ]:
daily_ab_testing['control_converted_rate'] = daily_ab_testing['control_converted'] / daily_ab_testing['control_visit'] 
daily_ab_testing['treatment_converted_rate'] = daily_ab_testing['treatment_converted'] / daily_ab_testing['treatment_visit'] 
daily_ab_testing

,date,control_visit,treatment_visit,control_converted,treatment_converted,control_converted_rate,treatment_converted_rate
0,2017-01-02,2859,2853,359,342,0.125568,0.119874
1,2017-01-03,9449,9471,1109,1095,0.117367,0.115616
2,2017-01-04,16027,16012,1911,1858,0.119236,0.116038
3,2017-01-05,22454,22517,2703,2606,0.120379,0.115735
4,2017-01-06,29060,29264,3465,3439,0.119236,0.117516
5,2017-01-07,35664,35873,4264,4207,0.11956,0.117275
6,2017-01-08,42351,42573,5059,5016,0.119454,0.117821
7,2017-01-09,48979,49188,5852,5797,0.11948,0.117854
8,2017-01-10,55633,55884,6603,6643,0.118689,0.118871
9,2017-01-11,62321,62557,7398,7411,0.118708,0.118468


In [ ]:
daily_ab_testing['p_value'] = daily_ab_testing.apply(lambda x: test_proportions_2indep(count1=x['treatment_converted'],
                                                     nobs1=x['treatment_visit'],
                                                     count2=x['control_converted'],
                                                     nobs2=x['control_visit'],
                                                     alternative='two-sided')[1], axis=1)
daily_ab_testing

,date,control_visit,treatment_visit,control_converted,treatment_converted,control_converted_rate,treatment_converted_rate,p_value
0,2017-01-02,2859,2853,359,342,0.125568,0.119874,0.512487
1,2017-01-03,9449,9471,1109,1095,0.117367,0.115616,0.707494
2,2017-01-04,16027,16012,1911,1858,0.119236,0.116038,0.374410
3,2017-01-05,22454,22517,2703,2606,0.120379,0.115735,0.127000
4,2017-01-06,29060,29264,3465,3439,0.119236,0.117516,0.520382
5,2017-01-07,35664,35873,4264,4207,0.11956,0.117275,0.344192
6,2017-01-08,42351,42573,5059,5016,0.119454,0.117821,0.461865
7,2017-01-09,48979,49188,5852,5797,0.11948,0.117854,0.430964
8,2017-01-10,55633,55884,6603,6643,0.118689,0.118871,0.924915
9,2017-01-11,62321,62557,7398,7411,0.118708,0.118468,0.895623


### Pada perhitungan sebelumnya, kita menegetahui bahwa power yang dimiliki sampai tanggal 24 Januari hanya sebesar 25% sehingga pada sesi ini mari kita coba cari jika power 80% berapa MDE yang akan bisa terdeteksi.

In [ ]:
n_obs = len(ab_test[ab_test['group'] == 'control']) + len(ab_test[ab_test['group'] == 'treatment'])
test_split = len(ab_test[ab_test['group'] == 'treatment']) / n_obs
test_split

0.5000636646764286

In [ ]:
# Membuat fungsi power untuk mengetahui relative effect berapa yang bisa d deteksi dengan power 80%
def power_result(daily_ab_testing):

  n_obs = len(ab_test[ab_test['group'] == 'control']) + len(ab_test[ab_test['group'] == 'treatment'])
  test_split = len(ab_test[ab_test['group'] == 'treatment']) / n_obs
  num_traffic = daily_ab_testing['control_visit'] + daily_ab_testing['treatment_visit']
  relative_effect = 0.05 #nilai MDE ini di coba coba sampai ditemukan power sebesar 80%


  power_result = power_proportions_2indep(diff=daily_ab_testing['control_converted_rate']*relative_effect,
                                               prop2=daily_ab_testing['control_converted_rate'], 
                                               nobs1=num_traffic*test_split,
                                               ratio=(1-test_split)/test_split,
                                               alpha=0.05,
                                               alternative='two-sided', udh
                                               return_results=False)
  return power_result

In [ ]:
# power_MDDE_1 artinya MDE nya 0.01
# power_MDE_2 artinya MDE nya 0.02 dst

daily_ab_testing['power_MDE_5'] = daily_ab_testing.apply(power_result, axis=1)
daily_ab_testing

,date,control_visit,treatment_visit,control_converted,treatment_converted,control_converted_rate,treatment_converted_rate,p_value,power_MDE_1,power_MDE_2,power_MDE_3,power_MDE_4,power_MDE_5
0,2017-01-02,2859,2853,359,342,0.125568,0.119874,0.512487,0.052342,0.059367,0.071122,0.087697,0.109186
1,2017-01-03,9449,9471,1109,1095,0.117367,0.115616,0.707494,0.057205,0.079032,0.116070,0.168757,0.236670
2,2017-01-04,16027,16012,1911,1858,0.119236,0.116038,0.374410,0.062459,0.100539,0.165580,0.257124,0.370546
3,2017-01-05,22454,22517,2703,2606,0.120379,0.115735,0.127000,0.067730,0.122304,0.215443,0.343253,0.492338
4,2017-01-06,29060,29264,3465,3439,0.119236,0.117516,0.520382,0.072807,0.143377,0.263044,0.421694,0.594117
5,2017-01-07,35664,35873,4264,4207,0.11956,0.117275,0.344192,0.078135,0.165544,0.312045,0.497904,0.683739
6,2017-01-08,42351,42573,5059,5016,0.119454,0.117821,0.461865,0.083452,0.187650,0.359549,0.566969,0.756530
7,2017-01-09,48979,49188,5852,5797,0.11948,0.117854,0.430964,0.088773,0.209707,0.405387,0.628874,0.814623
8,2017-01-10,55633,55884,6603,6643,0.118689,0.118871,0.924915,0.093812,0.230489,0.447015,0.680940,0.858031
9,2017-01-11,62321,62557,7398,7411,0.118708,0.118468,0.895623,0.099183,0.252480,0.489322,0.729742,0.894037


In [ ]:
# Membuat fungsi power untuk mengetahui power saat test di jalankan
def power_test(daily_ab_testing):

  n_obs = len(ab_test[ab_test['group'] == 'control']) + len(ab_test[ab_test['group'] == 'treatment'])
  test_split = len(ab_test[ab_test['group'] == 'treatment']) / n_obs
  num_traffic = daily_ab_testing['control_visit'] + daily_ab_testing['treatment_visit']
  relative_effect = (daily_ab_testing['control_converted'] - daily_ab_testing['treatment_converted'])/daily_ab_testing['control_converted']


  power_test = power_proportions_2indep(diff=daily_ab_testing['control_converted_rate']*relative_effect,
                                               prop2=daily_ab_testing['control_converted_rate'], 
                                               nobs1=num_traffic*test_split,
                                               ratio=(1-test_split)/test_split,
                                               alpha=0.05,
                                               alternative='two-sided',
                                               return_results=False)
  return power_test

In [ ]:
daily_ab_testing['power_test'] = daily_ab_testing.apply(power_test, axis=1)
daily_ab_testing

,date,control_visit,treatment_visit,control_converted,treatment_converted,control_converted_rate,treatment_converted_rate,p_value,power_MDE_1,power_MDE_2,power_MDE_3,power_MDE_4,power_MDE_5,power_test
0,2017-01-02,2859,2853,359,342,0.125568,0.119874,0.512487,0.052342,0.059367,0.071122,0.087697,0.109186,0.103017
1,2017-01-03,9449,9471,1109,1095,0.117367,0.115616,0.707494,0.057205,0.079032,0.116070,0.168757,0.236670,0.061497
2,2017-01-04,16027,16012,1911,1858,0.119236,0.116038,0.374410,0.062459,0.100539,0.165580,0.257124,0.370546,0.148451
3,2017-01-05,22454,22517,2703,2606,0.120379,0.115735,0.127000,0.067730,0.122304,0.215443,0.343253,0.492338,0.287113
4,2017-01-06,29060,29264,3465,3439,0.119236,0.117516,0.520382,0.072807,0.143377,0.263044,0.421694,0.594117,0.062786
5,2017-01-07,35664,35873,4264,4207,0.11956,0.117275,0.344192,0.078135,0.165544,0.312045,0.497904,0.683739,0.100717
6,2017-01-08,42351,42573,5059,5016,0.119454,0.117821,0.461865,0.083452,0.187650,0.359549,0.566969,0.756530,0.074072
7,2017-01-09,48979,49188,5852,5797,0.11948,0.117854,0.430964,0.088773,0.209707,0.405387,0.628874,0.814623,0.084185
8,2017-01-10,55633,55884,6603,6643,0.118689,0.118871,0.924915,0.093812,0.230489,0.447015,0.680940,0.858031,0.065973
9,2017-01-11,62321,62557,7398,7411,0.118708,0.118468,0.895623,0.099183,0.252480,0.489322,0.729742,0.894037,0.051490


In [ ]:
# save to excel dan download untuk kebutuhan visualisasi
from google.colab import files
daily_ab_testing.to_excel("daily_ab_testing_dengan_power_&_MDE.xlsx") 
files.download("daily_ab_testing_dengan_power_&_MDE.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### SELESAI